## Fetching data from *Open Data Platforms*: HIFLD Data
The Homeland Infrastructure Foundation-Level Data (HIFLD) has an Open Data Platform providing access to numerous useful datasets. [Open Data Platforms](https://www.esri.com/en-us/arcgis/products/arcgis-open-data) are becoming a popular platorm for hosting geospatial and non-geospatial data. Here we explore how to download those datasets via the ArcGIS Python API.

### The HIFLD Open Data Platform
If you navigate to the [HIFLD Open Data site](https://hifld-geoplatform.opendata.arcgis.com/) and search for "Electric Substations", you'll be taken to the Electric Substations Open Data access point: https://hifld-geoplatform.opendata.arcgis.com/datasets/electric-substations. Here, you'll see links to download the dataset in a choice of formats; you'll also see a link for the dataset's GeoService and GeoJSON APIs. The **GeoService API** is the ESRI REST endpoint for the data, a gateway for interacting or downloading the data. The **GeoJSON API** link will allow you to download the dataset directly in GeoJSON format. 

The code below outlines the steps for accessing the data via the GeoService API as it is more flexible, allowing us to query specific records if we want. First, however, we need to gather a bit of information from the GeoService API site:

### Navigating an Open Data Platform's *GeoService API* link
* Open the GeoService API [link](https://services1.arcgis.com/Hp6G80Pky0om7QvQ/arcgis/rest/services/Electric_Substations_1/FeatureServer/0/query?outFields=*&where=1%3D1). 
 * This site is a query result listing all the features. We want to back up a bit...
 
 
* At the top of the page are the sequence of links leading up to this query. Navigate backwards to the Substations link:<br>
https://services1.arcgis.com/Hp6G80Pky0om7QvQ/ArcGIS/rest/services/Electric_Substations_1/FeatureServer/0
 * This page shows information about the substations' **feature layer service**, revealing some useful information about the geoservice. 
 * Take note of the URL of this page. We can use this URL to access the data in Python using the ArcGIS Python API.

In [ ]:
#Import the arcgis FeatureLayer class
from arcgis.features import FeatureLayer

In [ ]:
#Create a FeatureLayer object, using the feature layer service's URL
serviceURL = 'https://services1.arcgis.com/Hp6G80Pky0om7QvQ/ArcGIS/rest/services/Electric_Substations_1/FeatureServer/0'
substations_Layer = FeatureLayer(serviceURL)

#Reveal the data type of the object created
type(substations_Layer)

The returned object is a an arcgis [FeatureLayer](https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#featurelayer); it allows us to interact with the spatial features stored on the server. Our objectives here are (1) to subset those substations falling within North Carolina, and then (2) save those records to a local shapefile. 

To subset the records, it helps to find out what fields are included in the feature layer, find what unique values are contained in that field, and then set up a query to extract the records we want in that dataset. 

Recall from previous examples, that a list of fields included in a feature layer is included in that feature layer's properties object. Once obtained, we can iterate through that list using a `for` loop, and print out each field's name.

Below are code examples showing how that's done. 

In [ ]:
#Get a list fields 
allFields = substations_Layer.properties.fields
print(len(allFields))

In [ ]:
#Now loop through each field in the "allFields" list and printing the field's name
for field in allFields:
    print(field.name)

We see that one of the fields is `STATE`. Let's see how each state name is listed in this dataset using the FeatureLayer's [`get_unique_values()`](https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html?highlight=get_unique#arcgis.features.FeatureLayer.get_unique_values) method. 

In [ ]:
#List the unique values of the STATE field
print(substations_Layer.get_unique_values('STATE'))

So, states are listed as their two digit abbreviation. Now we can [query](https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html?highlight=get_unique#arcgis.features.FeatureLayer.query) our FeatureLayer object to subset only records that fall within North Carolina. 

In [ ]:
#Subset North Carolina features
nc_Substations = substations_Layer.query("STATE = 'NC'")

In [ ]:
#Reveal the type of feature returned
type(nc_Substations)

The result of our query is a new type of object, a [FeatureSet](https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#featureset). The FeatureSet can easily be converted to a [Spatial Dataframe](https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#spatialdataframe) (via the [sdf](https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#arcgis.features.FeatureSet.sdf) function, which, in turn, as a function `to_featureclass()` that exports the data as a shapefile. See this [link](https://developers.arcgis.com/python/guide/introduction-to-the-spatially-enabled-dataframe/#Example:-Export-a-whole-dataset-to-a-shapefile:) for more info.

In [ ]:
#Convert the FeatureSet to a Spatial DataFrame
nc_Substations_sdf = nc_Substations.sdf
type(nc_Substations_sdf)

Now that we have a spatial dataframe, we can save it using the SpatialDataFrame's [to_featureclass()](https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#arcgis.features.SpatialDataFrame.to_featureclass) method.

In [ ]:
#Create a new folder to hold the feature class
import os
if not os.path.exists('HIFLD'):
    os.mkdir("HIFLD")

#Export the data to a feature class called "Substations.shp" in the HIFLD folder
nc_Substations_sdf.spatial.to_featureclass(location='./HIFLD/Substations.shp')

In [ ]:
#Zip the folder
import shutil
shutil.make_archive(base_name='HIFLD',format='zip',root_dir='./HIFLD')